In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import pickle
import pandas as pd

import time
from bs4 import BeautifulSoup
import requests
import json
import re
import tiktoken
import torch
import torch.nn.functional as F

from gensim.models import Word2Vec
import gensim
import gensim.downloader as api
from itertools import combinations

from datetime import datetime, timedelta
from langchain_ollama import OllamaLLM

In [5]:
def click_button(driver, timeout=10):
    try:
        # wait for button
        button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Pl')]"))
        )
        button.click()
    except TimeoutException:
        pass

def get_words(date=''):
    # webscraping
    options = Options()
    options.headless = True

    # init driver
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://www.nytimes.com/games/connections{date}')

    click_button(driver)

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//label[contains(@class, 'Card-module_label')]"))
    )
    words_elements = driver.find_elements(By.XPATH, "//label[contains(@class, 'Card-module_label')]")

    word_list = []
    for element in words_elements:
        time.sleep(0.5)
        if element.text.strip():
            word_list.append(element.text.strip())

    driver.quit()
    return word_list


In [31]:
wv = api.load('word2vec-google-news-300')

In [ ]:
# to start ollama
# OLLAMA_HOST=127.0.0.1:11435 ollama serve

words = get_words()
words = [word.lower() for word in words]

In [3]:


words = ['joker', 'dice', 'chance', 'casino', 'octopus', 'slot machine', 'cube', 'gamble', 'slice', 'bet', 'shiva', 'julienne', 'heat', 'venus de milo', 'risk', 'taxi driver']

In [7]:
print(words)

['bat', 'egg', 'love', 'witch', 'pumpkin', 'spider', 'blink', 'job', 'diablo', 'flutter', 'treasure', 'viper', 'scavenger', 'mustang', 'wink', 'darling']


In [6]:
prompt = "Given the following 16 words, order them into 4 groups of 4, based on their abstract relation to each other. for\
                example, group = fluster: faze, rattle, ruffle,throw. group = series of messages: chain, chat, conversation, thread\
                .group = places to lie/sit that move: cradle, hammock, rocker, swing. group = cylinder-shaped things: can, roller, silo, spool. \
                the words are 'joker', 'dice', 'chance', 'casino', 'octopus', 'slot machine', 'cube', 'gamble', 'slice', 'bet', 'shiva', 'julienne', 'heat', 'venus de milo', 'risk', 'taxi driver'"

In [7]:
llm = OllamaLLM(model="llama3.1")

response = llm.invoke(prompt)
print(response)

Here are the 16 words grouped into 4 categories based on their abstract relation:

**Group 1: Things related to Chance and Luck**

* Joker
* Dice
* Chance
* Gamble

These words all relate to concepts of chance, probability, and risk-taking.

**Group 2: Forms of Art or Objects inspired by them**

* Octopus (inspired by the art style of H.R. Giger, known for his biomechanical octopuses)
* Venus de Milo (a famous ancient Greek statue)
* Shiva (referring to the Hindu deity often depicted with multiple arms and faces)
* Julienne (likely referring to a French artist or style, but also possibly a type of vegetable cut into thin strips)

These words all relate to art, sculpture, or objects inspired by them.

**Group 3: Gaming and Casino-related Terms**

* Casino
* Slot machine
* Bet
* Chance

These words are all related to games of chance, casinos, and betting.

**Group 4: Concepts of Risk and Uncertainty**

* Heat (as in the heat of a situation or competition)
* Cube (referring to a die or a 

In [32]:


# create list of tuples of 4 words
tuples = {}

for tuple4 in combinations(words, 4):
    sim_sum = 0
    for w1, w2 in combinations(tuple4, 2):
        sim = 1- wv.similarity(w1, w2)
        sim_sum += sim

    tuples[tuple4] = sim_sum/4

sorted_tuples = dict(sorted(tuples.items(), key=lambda item: item[1], reverse=True))

# Initialize final tuples and a set for word membership checking
final_tuples = []
seen_words = set()

for key in sorted_tuples.keys():
    # Check if any word in the current tuple already exists in seen_words
    if all(word not in seen_words for word in key):
        final_tuples.append(key)
        seen_words.update(key)  # Add the words of the current tuple to the set

# Print the final tuples
print(final_tuples)

[('job', 'scavenger', 'mustang', 'wink'), ('bat', 'diablo', 'flutter', 'treasure'), ('pumpkin', 'blink', 'viper', 'darling'), ('egg', 'love', 'witch', 'spider')]


In [ ]:
d_model = 768       # embd dims
layer_norm_eps: float = 1e-5
d_vocab: int = 50257        # 50,000 mergres
n_ctx = 1024
d_head = 64     # d_model / d_head = num_heads
d_mlp = 3072
n_heads = 12
n_layers = 12

In [ ]:
# all we want to do is use the learnt information from training via attention on large amount of data 
# to select 4 words that belong in the same group (requires context, pop culture, references, jokes etc)

In [ ]:
# attention

class MultiHeadAttention():
    def __init__():
        super().__init__()

class SelfAttention()